In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DIR = "../input/medium-articles-dataset/medium_data.csv"

In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


import nltk 
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2

IMAGE_SIZE = (150, 150)

from skimage import io




In [ ]:



import re
df = pd.read_csv(DIR)

In [ ]:
df = df.drop(['url'], axis=1)



In [ ]:
df = df.dropna(subset = ['image'])

In [ ]:
df = df[:500]

In [ ]:
def image(x):
    try:
        if type(x) != str:
            return None
        
        img_path = "../input/medium-articles-dataset/images/" + x

        image = io.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, IMAGE_SIZE) 
        if type(image) == float:
            return None
        return image
    except:
#         img_path = "../input/medium-articles-dataset/images/1"

#         image = io.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = cv2.resize(image, IMAGE_SIZE) 
        return None
        
    
    
df.image = df.image.apply(lambda x: image(x))


In [ ]:
# df.image = df.image.replace(to_replace='None', value=np.nan).dropna()


df.image = df.image.dropna()


In [ ]:
text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def clean(text):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    return text
df.title = df.title.apply(lambda x: clean(x))
df.subtitle = df.subtitle.apply(lambda x: clean(x))

# df.responses  = df.responses.apply(lambda x: int(x))


In [ ]:
def cleaner(num):
    if (num > 500):
        return 500
    else:
        return num
df.claps = df.claps.apply(lambda x: cleaner(x))

In [ ]:
TRAIN_SIZE = 0.8
MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 20

train_data, test_data = train_test_split(df, test_size=1-TRAIN_SIZE,
                                         random_state=7) # Splits Dataset into Training and Testing set
print("Train Data size:", len(train_data))
print("Test Data size", len(test_data))

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.title)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.title),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.title),
                       maxlen = MAX_SEQUENCE_LENGTH)

subtitle_train = pad_sequences(tokenizer.texts_to_sequences(train_data.subtitle),
                        maxlen = MAX_SEQUENCE_LENGTH)
subtitle_test = pad_sequences(tokenizer.texts_to_sequences(test_data.subtitle),
                       maxlen = MAX_SEQUENCE_LENGTH)

responses_train = train_data.responses
responses_test = test_data.responses


reading_time_train = train_data.reading_time
reading_time_test = test_data.reading_time


y_train = train_data.claps
y_test = test_data.claps

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
img_train = train_data.image
img_test = test_data.image




In [ ]:
print(img_train.shape)

for i in img_train:
    print(i.shape)

In [ ]:
print(img_test.shape)

for i in img_test:
    print(i.shape)

In [ ]:
GLOVE_DIR = '../input/glove6b300dtxt/glove.6B.300d.txt'


embeddings_index = {}
f = open(GLOVE_DIR)
print('Loading GloVe from:', GLOVE_DIR,'...', end='')
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')
f.close()
print("Done.\n Proceeding with Embedding Matrix...", end="")

embedding_matrix = np.random.random((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(" Completed!")

In [ ]:
# img_train = np.array(img_train, dtype = 'float32')
# img_test = np.array(img_test, dtype = 'float32')
# img_train=np.asarray(img_train).astype(np.float32)
# img_test=np.asarray(img_test).astype(np.float32)

img_train=tf.convert_to_tensor(list(img_train), dtype = tf.float32)
img_test=tf.convert_to_tensor(list(img_test), dtype = tf.float32)


In [ ]:
#1
# from tensorflow.keras.layers import *


# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
# embedding_layer = Embedding(len(word_index) + 1,
#                            300,
#                            weights = [embedding_matrix],
#                            input_length = MAX_SEQUENCE_LENGTH,
#                            trainable=False,# prevent re-training the glove vector
#                            name = 'embeddings')
# embedded_sequences = embedding_layer(sequence_input)
# x = LSTM(60, return_sequences=True,name='lstm_layer')(embedded_sequences)
# x = GlobalMaxPool1D()(x)
# x = Dropout(0.1)(x)
# x = Dense(50, activation="relu")(x)
# x = Dropout(0.1)(x)
# preds = Dense(1)(x)


# ####

# from tensorflow.keras.models import Model

# model = Model(sequence_input, preds)
# model.compile(loss='mean_squared_error', optimizer='adam',
#              metrics = ['mae'])

In [ ]:
# model = keras.Sequential([
#       norm,
#       layers.Dense(64, activation='relu'),
#       layers.Dense(64, activation='relu'),
#       layers.Dense(1)
#   ])

#   model.compile(loss='mean_absolute_error',
#                 optimizer=tf.keras.optimizers.Adam(0.001))

In [ ]:
#1
from tensorflow.keras.layers import *

# Title
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedding_layer = Embedding(len(word_index) + 1,
                           300,
                           weights = [embedding_matrix],
                           input_length = MAX_SEQUENCE_LENGTH,
                           trainable=False,# prevent re-training the glove vector
                           name = 'embeddings')
embedded_sequences = embedding_layer(sequence_input)
x = LSTM(60, return_sequences=True,name='lstm_layer')(embedded_sequences)
# x = Conv1D(64, 5, activation='relu')(x)
x = GlobalMaxPool1D()(x)

x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
preds = Dense(10)(x)

#Subtitle
sequence_input2= Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedded_sequences2 = embedding_layer(sequence_input2)
x2 = LSTM(60, return_sequences=True,name='lstm_layer2')(embedded_sequences2)
# x = Conv1D(64, 5, activation='relu')(x)
x2 = GlobalMaxPool1D()(x2)
x2 = Dropout(0.1)(x2)
x2 = Dense(50, activation="relu")(x2)
x2 = Dropout(0.1)(x2)
preds2 = Dense(10)(x2)

#reading_time
input_y = Input(shape = (1,), dtype='int32')
y = Dense(2, activation="relu")(input_y)

#Image
input_image = Input(shape = (150,150,3)) # , dtype='object'
z = Conv2D(32, (3, 3), activation = 'relu')(input_image)
z = MaxPooling2D(2,2)(z)
z = Conv2D(32, (3, 3), activation = 'relu')(z)
z = MaxPooling2D(2,2)(z)
z = Flatten()(z)
z = Dense(128, activation="relu")(z)



concat = Concatenate()([preds, preds2, y, z])

output = Dense(1)(concat)

####

from tensorflow.keras.models import Model

model = Model(inputs = [sequence_input, sequence_input2, input_y, input_image], outputs = [output])

model.compile(loss='mean_absolute_error', optimizer='adam',
             metrics = ['mae'])

######

print('Training progress:')

history = model.fit([x_train,subtitle_train, reading_time_train, img_train], y_train, epochs = 15, batch_size=64, validation_data=([x_test,subtitle_test, reading_time_test, img_test], y_test))

mae = history.history['mae']
val_mae = history.history['val_mae']
epochs = range(1, len(mae)+1)

plt.plot(epochs, mae, label='Training mae')
plt.plot(epochs, val_mae, label='Validation MAE')
plt.title('Training and validation MAE')
plt.ylabel('MAE/val_MAE')
plt.xlabel('Epochs')
plt.legend()
plt.show();

In [ ]:
print (type(sequence_input))

In [ ]:
# #2 
# from tensorflow.keras.layers import *


# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# embedding_layer = Embedding(len(word_index) + 1,
#                            300,
#                            weights = [embedding_matrix],
#                            input_length = MAX_SEQUENCE_LENGTH,
#                            trainable=False,# prevent re-training the glove vector
#                            name = 'embeddings')
# embedded_sequences = embedding_layer(sequence_input)
# x = LSTM(60, return_sequences=True,name='lstm_layer')(embedded_sequences)
# x = Dense(64, activation="relu")(x)
# x = Dense(64, activation="relu")(x)
# x = Dense(64, activation="relu")(x)

# preds = Dense(10)(x)



# ####

# from tensorflow.keras.models import Model

# model = Model(sequence_input, preds)
# model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.001),
#              metrics = ['mae'])



# ######

# print('Training progress:')
# history = model.fit(x_train, y_train, epochs = 50, batch_size=64, validation_data=(x_test, y_test))

# mae = history.history['mae']
# val_mae = history.history['val_mae']
# epochs = range(1, len(mae)+1)

# plt.plot(epochs, mae, label='Training mae')
# plt.plot(epochs, val_mae, label='Validation MAE')
# plt.title('Training and validation MAE')
# plt.ylabel('MAE/val_MAE')
# plt.xlabel('Epochs')
# plt.legend()
# plt.show();

In [ ]:
tf.keras.utils.plot_model(model)


In [ ]:
test_predictions = model.predict([x_test,subtitle_test, reading_time_test, img_test]).flatten()

# a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [Claps]')
plt.ylabel('Predictions [Claps]')

In [ ]:
#1
mae = history.history['mae']
val_mae = history.history['val_mae']
epochs = range(1, len(mae)+1)

plt.plot(epochs, mae, label='Training mae')
plt.plot(epochs, val_mae, label='Validation MAE')
plt.title('Training and validation MAE')
plt.ylabel('MAE/val_MAE')
plt.xlabel('Epochs')
plt.legend()
plt.show();

In [ ]:
# print(x_test.shape)
# print(y_test.shape)
# test_predictions = model.predict(x_test)
# print(test_predictions.flatten().shape)
# print(test_predictions.shape)


# input input



In [ ]:
User_title = input("Your Title is: ")
User_subtitle = input("Your Subtitle is: ")


#User_title_pred = []
#User_subtitle_pred = []

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
User_title = re.sub(text_cleaning_re, ' ', str(User_title).lower()).strip()
User_subtitle = re.sub(text_cleaning_re, ' ', str(User_subtitle).lower()).strip()

User_title_tokenised = pad_sequences(tokenizer.texts_to_sequences(User_title),
                        maxlen = MAX_SEQUENCE_LENGTH)
User_subtitle_tokenised = pad_sequences(tokenizer.texts_to_sequences(User_subtitle),
                        maxlen = MAX_SEQUENCE_LENGTH)

User_title_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
User_subtitle_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

#User_title_pred.append(User_title_tokenised)
#User_subtitle_pred.append(User_subtitle_tokenised)


User_image_path = "../input/medium-articles-dataset/images/1.png"
User_image = io.imread(User_image_path)
User_image = cv2.cvtColor(User_image, cv2.COLOR_BGR2RGB)
User_image = cv2.resize(User_image, IMAGE_SIZE) 
User_image =tf.convert_to_tensor(list(User_image), dtype = tf.float32)
print(User_image.shape)

Reading_time = 10

User_title_input = Input(shape=(MAX_SEQUENCE_LENGTH,))
User_subtitle_input = Input(shape=(MAX_SEQUENCE_LENGTH,))
Reading_time_input = Input(shape = (1,))
User_image_input = Input(shape = (150,150,3)) # , dtype='object'


test_predictions_user = model.predict([User_title_input,User_subtitle_input, Reading_time_input, User_image_input])

print (test_predictions_user)